## Import the needed libraries 

In [1]:
import pandas as pd # Deal with the dataframe
import torch # Deal with neural network and tensors
import numpy as np # Deal with the Matrixies and arrays
import string # To Deal with the String object +]
import re # Regular expressions library to deal with pattern matching and string manipulation. 
import contractions # Library have most of the contractions 
import nltk # The main library to deal with NLP problems
from nltk.corpus import stopwords # To deal with stop words (contains list of stop words)
from nltk.tokenize import word_tokenize # To tokenize the sentences
import torch.nn as nn
import torch.optim as optim # To regonize the optimizer
import torch.utils.data as data
import math
import copy
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import dataset
from torch.utils.data import Dataset 
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence # Deal with the padding 
from sklearn.model_selection import train_test_split # To split data to test and training
import tqdm

In [3]:
# Read the dataset
df= pd.read_csv("train.csv")

In [4]:
df.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [6]:
# Check that all the col not equal 0
df["threat"].unique()

array([0, 1], dtype=int64)

In [7]:
# Check that all the col not equal 0
df["identity_hate"].unique()

array([0, 1], dtype=int64)

Deal with unbalnced data by getting a part of it which nearly balnced 

In [8]:
# Sum any record have 1 as in any class 
x = sum(df['toxic'] == 1)
print(x)
# Put any record that have all the 6 classes = 0 (true or false dataframe)
zeroes= (df["toxic"] == 0) & (df["severe_toxic"] == 0) & (df["obscene"] == 0) & (df["threat"] == 0) & (df["insult"] == 0) & (df["identity_hate"] == 0)
# Take the data that are not oll zeroes 
data_no_zero = df[np.logical_not(zeroes)]
#This will make the data more balanced by take the data that have one in its record and put some zeroes record as a weight for the zeroes in each class 
df = pd.concat([data_no_zero, df[zeroes].sample(n=5000, random_state=42)])

15294


## Cleaning Functions 

In [9]:
# Put the english stop words in a list
stop_words = set(stopwords.words('english'))

In [14]:
def cleaning_1st_step(record): # Function that hold the most basics cleaning functions
    # .lower() function ---> change all the chars into lower case 
    record = record.lower()
    
    # re (object from the library).sub() help to find the specific pattern we give it in each element and replacet with '' (nothing)  
    record = re.sub(r'https?://\S+', '',record )
    
    # library (contractions) .fix() function to change all the contractions to there expansion
    record = contractions.fix(record)
    
    # Delete with the punctuation
    record = record.translate(str.maketrans('', '',  string.punctuation))
    
    # Delete any additional space (more than one space)
    record = " ".join(record.split())
    
#     # Remove the stop word 
#     record = record.apply(lambda x: ' '.join([word for word in x.split() if word not in (stopWords)])
    # return the parameter after done all the above functions.
    return record

# The classes to built it from scratch

## Multi-Head-Attention

In [15]:
class MultiHeadAttention(nn.Module):
    # Give the constructor the dim of the model and the number of the head of the attention 
    def __init__(self, dim_model, no_heads):
        # Call the constructor of the super MultiHeadAttentionb class 
        super(MultiHeadAttention, self).__init__()
        
        # Check if the dim of the model divisible by the munber of the heads if yes do nothing but if no then send this 
        # sentence as an error
        # assert is a python keyword to get true or false result if true will continue the excution if false will send warning  
        assert dim_model % no_heads == 0, "dim of the model should be divisible by the number of the heads"
        
        # Declare the the dimention of the models
        self.dim_model = dim_model
        # Declare the number of the heads of the attention 
        self.no_heads = no_heads
        # floor division from divide the size of the model and the number of the heads
        self.d_k = dim_model // no_heads
        
        # Set square tensors with dims (dim of the model, dim of the model)
        # nn.linear(input to layer, output from layer) to build a netwaork layer 
        self.tensor_q = nn.Linear(dim_model, dim_model) # query layer
        self.tensor_k = nn.Linear(dim_model, dim_model) # key layer
        self.tensor_v = nn.Linear(dim_model, dim_model) # value layer
        self.tensor_output = nn.Linear(dim_model, dim_model) # the output layer
     
    # the Function to calculate the attention take (Q query , K key , v value and non mask)
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # torch.matmul() --> function to multiple the tensors 
        # (Query , the transpose of the key / square root of the d_k )
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None: # check if there is mask not equal none
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9) # fill the mask which = 0 and fill it with -1e9
        attn_probs = torch.softmax(attn_scores, dim=-1) # apply the softmax function 
        result = torch.matmul(attn_probs, V) # Multiply the softmax output and the value tensor 
        return result # return the output of the function 
     
    # function data to number of the heads 
    def split_heads(self, x):
        batch_size, seq_length, dim_model = x.size()
        #viwe() --> Function to change the the dim but keep the same informations 
        return x.view(batch_size, seq_length, self.no_heads, self.d_k).transpose(1, 2) # transpose(dim1,dim2)
    
    # start to combine the output from each head in this function.
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        # contigous() Help to compine the heads 
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.dim_model)
    
    # forward() function to call split_heads function to split the (query, key , value) to the heads then call 
    # scaled_dot_product_attention() function to apply the attention on it 
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.tensor_q(Q))
        K = self.split_heads(self.tensor_k(K))
        V = self.split_heads(self.tensor_v(V))
        
        # start the attention calculations 
        attention_result = self.scaled_dot_product_attention(Q, K, V, mask)
        # call the combine_heads() function to combine the attention that happen in each head and 
        # .tensor_output() --> function to help in further combination
        output = self.tensor_output(self.combine_heads(attention_result))
        return output

## Position wise feed forward network

In [16]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, dim_model, d_ff):
        # Call the constructor from the super PositionWiseFeedForward class  
        super(PositionWiseFeedForward, self).__init__()
        self.layer1 = nn.Linear(dim_model, d_ff) 
        self.layer2 = nn.Linear(d_ff, dim_model)
        self.relu = nn.ReLU() # Activation function

    def forward(self, x):
        # Give x (the input) to 1st layer then give it to the activation function then pass it into the 2 layer and 
        # pass it as the output of the function
        return self.layer2(self.relu(self.layer1(x)))

## Positional encoder

In [17]:
class PositionalEncoding(nn.Module):
    def __init__(self, dim_model, max_tokens_length):
        # called the constructor from the super PositionalEncoding class 
        super(PositionalEncoding, self).__init__()
        
        # Creat a tensor with dims (the max number of the unique tokens , size of the model)
        pe = torch.zeros(max_tokens_length, dim_model)
        # arrange the tensor and increase the flexibility to take diffrent size input
        position = torch.arange(0, max_tokens_length, dtype=torch.float).unsqueeze(1) 
        # 2nd part of the rule
        div_term = torch.exp(torch.arange(0, dim_model, 2).float() * -(math.log(10000.0) / dim_model))
        # sin(even postion * 2nd part of the rule)
        pe[:, 0::2] = torch.sin(position * div_term)
        # cos(odd positions * 2nd part of the rule)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        # Buffer are tonsor that pass with the model (while saving and laoding the model)
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        # function return the input + the positional encoder 
        return x + self.pe[:, :x.size(1)]

## Encoder Layer 

In [18]:
class EncoderLayer(nn.Module):
    def __init__(self, dim_model, num_heads, dim_ff, dropout):
        # Call the constructor from the super EncoderLayer class 
        super(EncoderLayer, self).__init__()
        # Take an object from the MultiHeadAttention() class 
        self.multi_attention = MultiHeadAttention(dim_model, num_heads)
        # Take an object from the PositionWiseFeedForward() class 
        self.feed_forward = PositionWiseFeedForward(dim_model, dim_ff)
        # Create 2 normalization layers 
        self.normal1 = nn.LayerNorm(dim_model)
        self.normal2 = nn.LayerNorm(dim_model)
        # Create a dropout layer to prevent the overfitting
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask): # Build the encoder 
        # Start 1st step in the encodeing which is Calculate the attention
        attn_output = self.multi_attention(x, x, x, mask)
        # Then Normalize the output from the attention layer 
        x = self.normal1(x + self.dropout(attn_output))
        
        # Then 2nd step we claculate the feed forward from the input 
        ff_output = self.feed_forward(x)
        # Then Normalize again and the step of the normalization should be repeated after each step
        x = self.normal2(x + self.dropout(ff_output))
        return x

## Decoder Layer 

In [19]:
class DecoderLayer(nn.Module):
    def __init__(self, dim_model, no_heads, dim_ff, dropout):
        # Call the constructor from super DecoderLayer() class 
        super(DecoderLayer, self).__init__()
        # Create object from the MultiHeadAttention() class 
        self.self_attention  = MultiHeadAttention(dim_model, no_heads)
        # Create another object from the MultiHeadAttention() class for the cross step between the outPut from the 
        # encoder and the multi-head attention in the decoder 
        self.cross_attention  = MultiHeadAttention(dim_model, no_heads)
        # Create an object from the PositionWiseFeedForward() class 
        self.feed_forward = PositionWiseFeedForward(dim_model, dim_ff)
        # Creat 3 layers of normalization layer same number as number of the steps in the decoder process
        self.normal1 = nn.LayerNorm(dim_model)
        self.normal2 = nn.LayerNorm(dim_model)
        self.normal3 = nn.LayerNorm(dim_model)
        # Create a dropout layer to prevent the overfitting by reducing co-adaptation between the units of the model
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, encoder_output, src_mask, target_mask): # Build the decoder 
        # 1st step --> start to calculate the Attention and the masking
        attention_output = self.self_attention(x, x, x, target_mask)
        # then Normalize the output 
        x = self.normal1(x + self.dropout(attention_output))
        # 2nd step --> Start to do the attention process on the encoder output and the output from the Multi-head 
        # attention in the decoder (1st step in the decoder) 
        attention_output = self.cross_attention (x, encoder_output, encoder_output, src_mask)
        # Then normalize again
        x = self.normal2(x + self.dropout(attention_output))
        # 3rd step Calculate the feed forward 
        ff_output = self.feed_forward(x)
        # Last Normalize again
        x = self.normal3(x + self.dropout(ff_output))
        return x

## Transformer Model

In [20]:
class Transformer(nn.Module):
    def __init__(self, src_unique_tokens_size, target_unique_tokens_size, dim_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        # Start to embedding the source and the target 
        # nn.embedding () --> function take (the number of the unique tokens , the dim of the embedding vector)
        self.encoder_embedding = nn.Embedding(src_unique_tokens_size, dim_model)
        self.decoder_embedding = nn.Embedding(target_unique_tokens_size, dim_model)
        # Create an object from the PositionalEncoding() class and give it the model dim and the max number of tokens 
        self.positional_encoding = PositionalEncoding(dim_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(dim_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(dim_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(dim_model, target_unique_tokens_size)
        # Relu--> 
        # 2nd linear layer----> output
        self.dropout = nn.Dropout(dropout)

    # Masking 
    def generate_mask(self, src, target):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        target_mask = (target != 0).unsqueeze(1).unsqueeze(3)
        seq_length = target.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        nopeak_mask = nopeak_mask.to(torch.device('cuda'))
        target_mask = target_mask & nopeak_mask
        return src_mask, target_mask

    
    def forward(self, src, target):
        src_mask, target_mask = self.generate_mask(src, target)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        target_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(target)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = target_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, target_mask)
        
        output = self.fc(dec_output) #
        # output self.fc2(output) # 2nd layer 
        return output

### Start with Masking then embeding the data that will go in calc in the positional encoder 
### After that we do the encoder layer ( multi attention and Feed forward) 
### Take the output from the encoder layer to the decoder layer and make a cross attention and the feed forward 

## Start to split the data 

In [21]:
target= df.iloc[:, 2:8]
target.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,1,1,1,0,1,0
12,1,0,0,0,0,0
16,1,0,0,0,0,0
42,1,0,1,0,1,1
43,1,0,1,0,1,0


In [22]:
# Split the data into (tarin,test) and each to (src,target)
x_train, x_test, y_train, y_test = train_test_split(df['comment_text'] , target , test_size=0.8, random_state=0)

In [23]:
# change the train col into dataframe to have a col name 
train_x =  x_train.to_frame()

In [24]:
y_train

,toxic,severe_toxic,obscene,threat,insult,identity_hate
58787,1,1,1,0,1,0
415,1,0,1,0,1,0
71909,1,0,1,0,1,0
78991,1,0,1,0,1,0
45237,0,0,0,0,0,0
...,...,...,...,...,...,...
128843,1,0,0,0,0,0
150971,0,0,0,0,0,0
96455,1,0,0,0,0,0
105991,1,0,0,0,0,0


In [25]:
train_x

,comment_text
58787,"Fuck, Shit, Damn, Cock, Pissant, Motherfucker,..."
415,Thank you for your RACIST experimenting with t...
71909,"Don't be a Dick\nYou know what you did, don't ..."
78991,STOP CHANGON IT \n\nOR I AM TWIST OFF YOURE LI...
45237,"""\n\n Nonexistent Image in """"Name"""" \n\nThe im..."
...,...
128843,WHY DID YOU EDIT THE ARTICLE ON KURDISTAN? \n\...
150971,"""\nAww thanks -) from PC 28 at Stevenage libra..."
96455,""":: I don't really want to deal with an amoeba..."
105991,AVRUCH you truly do show your colours.....this...


In [26]:
# Then concat the the src and target of the training 
train_data = pd.concat([train_x , y_train], axis=1)

In [27]:
train_data

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
58787,"Fuck, Shit, Damn, Cock, Pissant, Motherfucker,...",1,1,1,0,1,0
415,Thank you for your RACIST experimenting with t...,1,0,1,0,1,0
71909,"Don't be a Dick\nYou know what you did, don't ...",1,0,1,0,1,0
78991,STOP CHANGON IT \n\nOR I AM TWIST OFF YOURE LI...,1,0,1,0,1,0
45237,"""\n\n Nonexistent Image in """"Name"""" \n\nThe im...",0,0,0,0,0,0
...,...,...,...,...,...,...,...
128843,WHY DID YOU EDIT THE ARTICLE ON KURDISTAN? \n\...,1,0,0,0,0,0
150971,"""\nAww thanks -) from PC 28 at Stevenage libra...",0,0,0,0,0,0
96455,""":: I don't really want to deal with an amoeba...",1,0,0,0,0,0
105991,AVRUCH you truly do show your colours.....this...,1,0,0,0,0,0


In [28]:
# start to change the test part too into dataframe 
test_x = x_test.to_frame()

In [29]:
# Then concat the test src and target for the testing 
test_data = pd.concat([test_x , y_test], axis=1)

In [30]:
test_data.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
59210,User Jkelly is incapable of behaving responsib...,1,0,0,0,1,0
7668,Ashwamedha requires a master of ceremony (yaja...,1,0,0,0,0,0
111073,Are you complaining about the page stating him...,1,0,0,0,1,0
149208,into a Jewish family,0,0,0,0,0,0
66151,"""\n\nThe piece of classical strings music that...",0,0,0,0,0,0


In [ ]:
def data_process(dataset, vocab, tokenizer):
    """Converts raw text into a flat Tensor."""
    data = []
    for item in dataset:
            data.append(torch.tensor(vocab(tokenizer(item)), dtype=torch.long))
        
    if len(data) == 0:
        return torch.tensor([0.,3.])
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

def pad(text):
    if text.shape[0] > 300:
        return text
    padding = torch.zeros(300 - text.shape[0])
    return torch.cat((text, padding))


class Customtrain(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe.reset_index()
        # preprocessing
        self.data["comment_text"] = self.data["comment_text"].apply(cleaning_1st_step)
        
        ########
        self.tokenizer = get_tokenizer('basic_english')
        self.vocab = build_vocab_from_iterator(map(self.tokenizer, self.data['comment_text']), specials=['<unk>'])
        self.vocab.set_default_index(self.vocab['<unk>'])
      


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data['comment_text'].iloc[idx] 
        text = data_process(text, self.vocab, self.tokenizer).long() # Start to tokenize 
        
        text = pad(text) # Padding
       
        if text.shape[0] > 300:    
            text = text[ :300]
            
        #text = text.view(1,-1)
        label = self.data.iloc[idx, 2:8]
        label= torch.tensor(label)
        return text , label

In [32]:
train_data.iloc[2,2:8]

severe_toxic     0
obscene          1
threat           0
insult           1
identity_hate    0
Name: 71909, dtype: object

In [33]:
train_data.head(20)

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
58787,"Fuck, Shit, Damn, Cock, Pissant, Motherfucker,...",1,1,1,0,1,0
415,Thank you for your RACIST experimenting with t...,1,0,1,0,1,0
71909,"Don't be a Dick\nYou know what you did, don't ...",1,0,1,0,1,0
78991,STOP CHANGON IT \n\nOR I AM TWIST OFF YOURE LI...,1,0,1,0,1,0
45237,"""\n\n Nonexistent Image in """"Name"""" \n\nThe im...",0,0,0,0,0,0
159281,LoL!! \n\nyou're GAY!! you will never know how...,1,1,1,0,1,1
36593,User:NHRHS2010 is a homo like mitt romney is. ...,1,0,0,0,0,0
83419,"So in other words, WP is so riddled with troll...",1,0,0,0,0,0
46934,"Well, user Burgass has done it again. Just loo...",0,0,0,0,0,0
47631,Fuck you. I'll just make another account. You'...,1,1,1,0,1,0


### Do Data custimization

In [34]:
training_data =Customtrain(train_data)

In [35]:
testing_data =Customtrain(test_data)

## Start to dataloader

In [36]:
train_dataloader = DataLoader(training_data, batch_size=1, shuffle=False)

In [51]:
test_dataloader = DataLoader(testing_data, batch_size=1, shuffle=False)

## Set the inputs 

In [52]:
vocab_size = 136951  # Vocabulary size
d_model = 512  # Embedding dimension
nhead = 8  # Number of attention heads
num_layers = 6  # Number of encoder layers
d_ff = 2048  # Feedforward dimension
max_seq_length = 300  # Maximum sequence length
num_classes = 6 # Number of classes for classification
dropout = 0.1  

In [39]:
model = Transformer(vocab_size,num_classes, d_model, nhead, num_layers,d_ff, max_seq_length, dropout)

In [40]:
# Choose Adam optimizer with learning rate 
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [41]:
criterion =  nn.BCEWithLogitsLoss()

In [42]:
import torchtext
def train_model(model, train_dataloader , optimizer, device, num_epochs):
    device=torch.device('cuda')
    model=model.to(device)
    # Loop through the specified number of epochs
    for epoch in range(num_epochs):
        # Set the model to training mode
        model.train()
        # Initialize total loss for the current epoch
        total_loss = 0
        correct_predictions = 0
        total_predictions = 0
 
        # Loop through the batches in the training data
        for text , label in tqdm.tqdm(train_dataloader) :
            # There is some record give error so ignore it by try and except 
            try:
                text= text.long().to(device)
                label= label.long().to(device)
                optimizer.zero_grad() # Call the optimizer
            
            
        # Start to train the model and see the predictions and then update the loss and the learning rate in the optimizer
                outputs = model(text, label) 
                loss =criterion(outputs, label.long())  
                total_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                correct_predictions += (predicted == label).sum().item()
                total_predictions += label.size(0)
                loss.backward() 
                optimizer.step()
            except:
                continue

In [43]:
# from numba import cuda
#device = cuda.select_device(0)
#device = "cuda" if torch.cuda.is_available() else "cpu"
#device

In [44]:
# Clear all the cashes 
torch.cuda.empty_cache()

In [45]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:3431'


In [46]:
# Start the Tarining
train_model(model, train_dataloader, optimizer, 'cuda', num_epochs=5)

100%|██████████████████████████████████████████████████████████████████████████████| 4245/4245 [01:38<00:00, 43.22it/s]


In [49]:
import torchtext
def test_model(model, test_dataloader , device, num_epochs):
    device=torch.device('cuda')
    model=model.to(device)
    # Loop through the specified number of epochs
    for epoch in range(num_epochs):
        # Set the model to training mode
        model.train()
        # Initialize 2 varubles
        correct_predictions = 0
        total_predictions = 0
 
        # Loop through the batches in the training data
        for text , label in tqdm.tqdm(test_dataloader) :
            try:
                text= text.long().to(device)
                label= label.long().to(device)
                
                outputs = model(text, label)
                output.squeeze(1) # use it to squeeze the dim
                for text , label in zip(output, label):
                    correct_predictions +=1 
                
            except:
                continue
    # Return the accurcy ratio 
    return correct_predictions/(len(test_loader)*6)

In [53]:
# Start the test and Get the accurcy 
accurcy=test_model(model, test_dataloader, 'cuda', num_epochs=1)

100%|████████████████████████████████████████████████████████████████████████████| 16980/16980 [06:35<00:00, 42.90it/s]


NameError: name 'test_loader' is not defined

In [ ]:
# Print the accury ratio
print(accurcy)